In [1]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/msmarco-bert-base-dot-v5')

ImportError: cannot import name 'LayerNormalization' from 'tensorflow.python.keras.layers.normalization' (/home/jayr/.local/lib/python3.8/site-packages/tensorflow/python/keras/layers/normalization/__init__.py)

In [5]:
import xml.etree.ElementTree as ET

tree_dev = ET.parse('./data/COPA-resources/datasets/copa-dev.xml')
root_dev = tree_dev.getroot()


In [35]:
all_docs = []

for item in root_dev.findall('item'):
    # print(item.attrib)
    doc = {
        "id":item.attrib['id'],
        "data": {
            "asks-for": item.attrib["asks-for"],
            "most-plausible-alternative": item.attrib['most-plausible-alternative'],
            "p" : item.find("p").text,
            "1": item.find("a1").text,
            "2": item.find("a2").text
        },
        "contents": "Context: {0} Alternative 1: {1} Alternative 2: {2}".format(item.find("p").text, item.find("a1").text,item.find("a2").text)
    }
    all_docs.append(doc)
len(all_docs)

500

In [36]:
all_docs_passages = [doc['contents'] for doc in all_docs]
all_docs_passages[0]

'Context: My body cast a shadow over the grass. Alternative 1: The sun was rising. Alternative 2: The grass was cut.'

In [37]:
all_docs_embeddings = model.encode(all_docs_passages, show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [11]:
import xml.etree.ElementTree as ET

tree = ET.parse('./data/COPA-resources/datasets/copa-test.xml')
root = tree.getroot()


In [50]:
from tqdm import tqdm
import numpy as np

kshot = 3

f = open("results/copa/text_davinci_002_sbert_bertv5_{0}shot.test".format(kshot),'w')

logs = []

for item in tqdm(root.findall('item')):
    qid = item.attrib['id']
    p = item.find("p").text
    # print(p)
    a1 = item.find("a1").text
    a2 = item.find("a2").text

    query_embedding = model.encode("Context: {0} Alternative 1: {1} Alternative 2: {2}".format(p,a1,a2))

    topk = util.dot_score(query_embedding, all_docs_embeddings)[0].topk(kshot)
    
    hits = np.array(all_docs)[topk.indices]

    # print(hits)


    
    # hits = searcher.search(p, k=kshot)
    # # print(p)
    prompt = ""
    for hit in hits:
    #     hit = json.loads(hit.raw)
        template = "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative {3}\n\n"
        prompt += template.format(hit['data']['p'],hit['data']['1'],hit['data']['2'],hit['data']['most-plausible-alternative'])
    
    prompt += "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative".format(p,a1,a2)
    # print(prompt)
    res = generate(prompt)
    # print(res)
    
    towrite = "{0} {1} {2}".format(qid,"1" if res == "1" else "0","1" if res == "2" else "0")
    f.write(towrite+"\n")

f.close()
    
        # print(hit['data'])


100%|██████████| 500/500 [02:51<00:00,  2.92it/s]
